This code grabs images from a Wikipedia page and creates an image collage with them

Import the libraries

In [2]:
from PIL import Image, ImageDraw
from bs4 import BeautifulSoup
import requests
from io import BytesIO

Call a URL and write to a BeautifulSoup object

In [59]:
url = 'https://en.wikipedia.org/wiki/Chicago_Bears'
##URL is called directly
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

Find the first div tag that has the specified class and save it to a variable

In [60]:
get_body = soup.body.find('div', attrs={'class':'mw-content-ltr mw-parser-output'})

Remove tthe role=presentation tag because those are Wikipdia-specific page elements

In [61]:
# Remove the tags with role=presentation
for tag in get_body.find_all(attrs={"role": ["presentation", "navigation"]}):
    tag.decompose()

In [62]:
#Find all images in body
img_tags = get_body.find_all('img')

Remove the data-file-height attribute because it keeps us from finding the pictures with a height and width attribute we specifiy in the next code block

In [63]:
# Remove the data-file-height attribute
for tag in img_tags:
    if 'data-file-height' in tag.attrs:
        del tag['data-file-height']
        del tag['data-file-width']

Since height and width under 20 is likely a wikipedia-specific logo, we can filter that out
However, some images, do not have a height or width tag, in that case we can skip that tag

In [64]:
filtered_images = []
# Remove <img> tags with both height and width less than 20px
for img in img_tags:
    width = img.get('width')
    height = img.get('height')
    # Ensure width and height are both present and can be converted to integers
    if width is not None and height is not None:
        try:
            width = int(width)
            height = int(height)
            if width > 20 and height > 20:
                filtered_images.append(img)
        except ValueError:
            # In case width or height are not valid integers, skip this tag
            continue

In [ ]:
# print(filtered_images)
for i in range(5):
    print(filtered_images[i])

In [ ]:
#Grab the URL from the images and write them to list
image_urls = []
for img in filtered_images:
    url = 'https:' + img['src']
    image_urls.append(url)

print(image_urls)

In [67]:
#Download the Images
images = []
for url in image_urls:
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    images.append(img)

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x0000023E302B47C0>

In [68]:
# Set the desired size for all images
desired_width = 220
desired_height = 220

resized_images = [img.resize((desired_width, desired_height), Image.LANCZOS) for img in images]

In [70]:
# Determine the collage dimensions
cols = 3
rows = (len(resized_images) + cols - 1) // cols
collage_width = cols * desired_width
collage_height = rows * desired_height

# Create a new image for the collage
collage = Image.new('RGBA', (collage_width, collage_height))

# Paste the resized images into the collage
for i, img in enumerate(resized_images):
    x = (i % cols) * desired_width
    y = (i // cols) * desired_height
    collage.paste(img, (x, y))

# Show and save the collage
collage.show()
collage.save('collage.jpg')


OSError: cannot write mode RGBA as JPEG